## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key
#Configure gmaps
gmaps.configure(api_key=g_key)

In [28]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Wad Rawah,SD,15.1603,33.1397,76.37,75,100,11.12,overcast clouds
1,1,Lebu,CL,-37.6167,-73.6500,52.50,95,100,9.53,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,33.46,64,75,12.66,broken clouds
3,3,Khatanga,RU,71.9667,102.5000,57.83,75,100,8.32,overcast clouds
4,4,Dawlatabad,AF,36.4141,64.9053,82.78,21,40,4.81,scattered clouds


In [29]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Wad Rawah,SD,15.1603,33.1397,76.37,75,100,11.12,overcast clouds
4,4,Dawlatabad,AF,36.4141,64.9053,82.78,21,40,4.81,scattered clouds
7,7,Tuktoyaktuk,CA,69.4541,-133.0374,77.00,44,1,16.11,clear sky
9,9,Celestun,MX,20.8667,-90.4000,84.38,71,4,14.99,clear sky
10,10,Sibolga,ID,1.7427,98.7792,76.06,86,83,2.95,broken clouds
12,12,Hithadhoo,MV,-0.6000,73.0833,81.10,78,95,8.05,overcast clouds
18,18,Mayumba,GA,-3.4320,10.6554,71.51,89,20,8.16,few clouds
19,19,Mahebourg,MU,-20.4081,57.7000,71.26,64,75,17.27,broken clouds
21,21,Isangel,VU,-19.5500,169.2667,82.51,74,20,5.75,few clouds
25,25,Elko,US,41.0002,-115.5012,88.74,24,64,10.00,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
# 4b. Drop any empty rows and create a new DataFrame that doesn't have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how = 'any')
preferred_cities_df.count()

City_ID                247
City                   247
Country                247
Lat                    247
Lng                    247
Max Temp               247
Humidity               247
Cloudiness             247
Wind Speed             247
Current Description    247
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Wad Rawah,SD,76.37,overcast clouds,15.1603,33.1397,
4,Dawlatabad,AF,82.78,scattered clouds,36.4141,64.9053,
7,Tuktoyaktuk,CA,77.00,clear sky,69.4541,-133.0374,
9,Celestun,MX,84.38,clear sky,20.8667,-90.4000,
10,Sibolga,ID,76.06,broken clouds,1.7427,98.7792,
12,Hithadhoo,MV,81.10,overcast clouds,-0.6000,73.0833,
21,Isangel,VU,82.51,few clouds,-19.5500,169.2667,
25,Elko,US,88.74,broken clouds,41.0002,-115.5012,
26,Sabla,NG,84.16,clear sky,12.7525,4.7077,
28,Thompson,CA,75.36,few clouds,55.7435,-97.8558,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        params["location"] = f"{lat},{lng}"
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        except (IndexError):
            print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
# 7. Drop any empty rows and create a new DataFrame that doesn't have empty rows.
hotel_df = hotel_df.dropna(how = 'any')
hotel_df.count()

City                   247
Country                247
Max Temp               247
Current Description    247
Lat                    247
Lng                    247
Hotel Name             247
dtype: int64

In [19]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv
clean_hotel.to_csv("clean_hotel_df.csv", index_label="City_ID")

In [20]:
clean_hotel = hotel_df.loc[hotel_df['Hotel Name'] != '']

In [21]:
clean_hotel.loc[clean_hotel['Hotel Name']=='']

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [24]:
hotel_df = clean_hotel
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Wad Rawah,SD,76.37,overcast clouds,15.1603,33.1397,ود الترابي
4,Dawlatabad,AF,82.78,scattered clouds,36.4141,64.9053,WASIM ASGAR KHAN
7,Tuktoyaktuk,CA,77.00,clear sky,69.4541,-133.0374,End of The Road Ltd.
9,Celestun,MX,84.38,clear sky,20.8667,-90.4000,Posada Lilia
10,Sibolga,ID,76.06,broken clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
12,Hithadhoo,MV,81.10,overcast clouds,-0.6000,73.0833,Scoop Guest House
21,Isangel,VU,82.51,few clouds,-19.5500,169.2667,Tanna Lodge
28,Thompson,CA,75.36,few clouds,55.7435,-97.8558,Burntwood Hotel
32,Atuona,PF,77.40,scattered clouds,-9.8000,-139.0333,Villa Enata
36,Jamestown,US,80.24,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
pwd


'C:\\Users\\jacqu\\Desktop\\Class Folder\\Vacation_Search'